In [29]:
import os

import folium
from folium import Map, Marker, LayerControl
#from folium.plugins import MarkerCluster
from folium.plugins import BeautifyIcon
import json


import fiona as fn
import shapely
from shapely import geometry
from shapely.geometry import shape, Polygon, mapping, MultiPolygon, LineString, Point
from shapely.ops import cascaded_union, transform
import pandas as pd

import openrouteservice as ors

from openrouteservice import client

from openrouteservice import convert

import pyproj

import pandas as pd
import geopandas as gpd

#----





token = "5b3ce3597851110001cf6248910f9ec899b94de0b25cb154d853f0be"
sites_csv = "./pts.csv"

incidents_json = "./incidents.json"

depot = [-12.195997, -77.01355]


In [30]:
deliveries_data = pd.read_csv(sites, index_col="index")
deliveries_data.head()

,lon,lat
index,,
0,-77.013559,-12.195996
1,-76.997766,-12.191298
2,-76.988325,-12.164031
3,-76.993475,-12.162436
4,-76.978025,-12.192305


In [ ]:
# Define the vehicles
# https://openrouteservice-py.readthedocs.io/en/latest/openrouteservice.html#openrouteservice.optimization.Vehicle
vehicles = list()
for idx in range(3):
    vehicles.append(
        ors.optimization.Vehicle(
            id=idx, 
            start=list(reversed(depot)),
            end=list(reversed(depot)),
            capacity=[900],
            #time_window=[1553241600, 1553284800]  # Fri 8-20:00, expressed in POSIX timestamp
        )
    )
    
# Next define the delivery stations
# https://openrouteservice-py.readthedocs.io/en/latest/openrouteservice.html#openrouteservice.optimization.Job
deliveries = list()
for delivery in deliveries_data.itertuples():
    deliveries.append(
        ors.optimization.Job(
            id=delivery.Index,
            location=[delivery.lon, delivery.lat],
            service=1200,  # Assume 20 minutes at each site
            amount=[300],
            #time_windows=[[
            #    int(delivery.Open_From.timestamp()),  # VROOM expects UNIX timestamp
            #    int(delivery.Open_To.timestamp())
            #]]
        )
    )

In [ ]:
# Initialize a client and make the request
ors_client = ors.Client(key=token)  # Get an API key from https://openrouteservice.org/dev/#/signup
opt_result = ors_client.optimization(
    jobs=deliveries,
    vehicles=vehicles,
    geometry=True
)
for idx, rt in enumerate(opt_result['routes']):
    #geometry = ors.convert.decode_polyline(rt['geometry'])
    opt_result['routes'][idx]['geometry'] = ors.convert.decode_polyline(rt['geometry'])

In [33]:
# Make a map!

# First define the map centered around Beira
m = folium.Map(location=depot, tiles='OpenStreetMap', zoom_start=12)    


# Plot the locations on the map with more info in the ToolTip
for location in deliveries_data.itertuples():
    #tooltip = folium.map.Tooltip("<h4><b>ID {}</b></p><p>Supplies needed: <b>{}</b></p>".format(
    #    location.Index, location.Needed_Amount
    #))
    
    folium.Marker(
        location=[location.lat, location.lon],
        #tooltip=tooltip,
        icon=BeautifyIcon(
            icon_shape='marker',
            #number=int(location.index),
            spin=True,
            text_color='red',
            background_color="#FFF",
            inner_icon_style="font-size:12px;padding-top:-5px;"
        )
    #)
    ).add_to(m)    
    
# The vehicles are all located at the port of Beira
#depot = [-19.818474, 34.835447]

folium.Marker(
    location=depot,
    icon=BeautifyIcon(
            icon_shape='marker',
            #number=int(location.index),
            spin=True,
            text_color='green',
            background_color="pink",
            inner_icon_style="font-size:12px;padding-top:-5px;"
        ),
    setZIndexOffset=1000
).add_to(m)


# Add the output to the map
for color, route in zip(['green', 'red', 'blue'], opt_result['routes']):
    
    #decoded=ors.convert.decode_polyline(route['geometry'])  # Route geometry is encoded
    #print(decoded)
    
       
    gj = folium.GeoJson(
        name='Vehicle {}'.format(route['vehicle']),
        data={"type": "FeatureCollection", "features": [{"type": "Feature", 
                                                         "geometry": route['geometry'],
                                                         "properties": {"color": color}
                                                        }]},
        style_function=lambda x: {"color": x['properties']['color']}
    )
    gj.add_child(folium.Tooltip(
        """<h4>Vehicle {vehicle}</h4>
        <b>Distance</b> {distance} m <br>
        <b>Duration</b> {duration} secs
        """.format(**route)
    ))

    gj.add_to(m)
    
i2 = folium.features.GeoJson(open(incidents_json).read())
i2.add_to(m)


folium.LayerControl().add_to(m)
m

In [19]:
rts = []

for idx, rt in enumerate(opt_result['routes']):
                                             
    rts.append({
        "type": "Feature", 
        "geometry": rt['geometry'],
        "properties": {"idx": idx,
                       "vehicle": rt['vehicle']}
    })
    
rts = gpd.GeoDataFrame.from_features(rts)
rts.crs = {'init' :'epsg:4326'}

In [34]:
# Load geojson with incidents
#ncidents_file = "https://raw.githubusercontent.com/python-visualization/folium/master/examples/data/antarctic_ice_edge.json"
incidents = gpd.read_file("./incidents.json")
incidents.crs = {'init' :'epsg:4326'}

In [24]:
# Computer Intersections

crosses = gpd.sjoin(incidents, rts, how='left', op='intersects', lsuffix='left', rsuffix='right')
crosses.head()

,stroke,stroke-width,stroke-opacity,fill,fill-opacity,incident,geometry,index_right,idx,vehicle
0,#555555,2,1,#555555,0.5,flood_zone,POLYGON ((-76.96862697601318 -12.1659603049577...,1,1,1
1,#555555,2,1,#555555,0.5,zombie_attack,POLYGON ((-76.99416160583495 -12.1957651872979...,2,2,2
2,#555555,2,1,#555555,0.5,flood_zone,POLYGON ((-76.98965549468994 -12.1925142896104...,2,2,2


In [26]:
# Get Unique indexes that need to be recalculated

bad_rts = pd.unique(crosses['idx'])
bad_rts

array([1, 2])

In [ ]:
# Now we know that we have to recalculate the route for vehicles that cross an issue area
bad_rts = 

In [ ]:
# Visualize start and destination point on map
coordinates = [[11.653361, 52.144116], [11.62847, 52.1303]] # Central Station and Fire Department
for coord in coordinates:
    folium.map.Marker(list(reversed(coord))).add_to(map_tweet)

# Regular Route
avoided_point_list = [] # Create empty list with avoided tweets
route_directions = CreateRoute(avoided_point_list) # Create regular route with still empty avoided_point_list

folium.features.GeoJson(data=route_directions,
                        name='Regular Route',
                        style_function=style_function('#ff5050'),
                        overlay=True).add_to(map_tweet)
print('Generated regular route.')


In [ ]:
coordinates = [[11.653361, 52.144116], [11.62847, 52.1303]]
route_request = {'coordinates': coordinates, 
                    'format_out': 'geojson',
                    'profile': 'driving-car',
                    'preference': 'shortest',
                    'instructions': False
                }

#'options': {'avoid_polygons': geometry.mapping(MultiPolygon(avoided_point_list))}} 
route_directions = ors_client.directions(**route_request)
route_directions

In [ ]:
result['routes'][0]['geometry'] = decoded=ors.convert.decode_polyline(result['routes'][0]['geometry'])
result['routes'][0]

In [ ]:
# Check if flood affected tweet is located on route
try:
    for site_poly in flood_tweets:
        poly = Polygon(site_poly)
        if poly.within(dilated_route):
            avoided_point_list.append(poly)

            # Create new route and buffer
            route_directions = CreateRoute(avoided_point_list, 1)
            dilated_route = CreateBuffer(route_directions)

    folium.features.GeoJson(data=route_directions,
                            name='Alternative Route',
                            style_function=style_function('#006600'),
                            overlay=True).add_to(map_tweet)
    print('Generated alternative route, which avoids affected areas.')
except Exception: 
    print('Sorry, there is no route available between the requested destination because of too many blocked streets.')

In [ ]:
# Later
# Function to create buffer around requested route
def CreateBuffer(route_directions): 
    line_tup = []
    for line in route_directions['features'][0]['geometry']['coordinates']:
        tup_format = tuple(line)
        line_tup.append(tup_format)  

    new_linestring = LineString(line_tup)
    dilated_route = new_linestring.buffer(0.001)
        
    return dilated_route